<a href="https://colab.research.google.com/github/akhilesh-kanmanthreddy/2024_Science_Fair_Tumor_Evolution_Forecasting/blob/main/2024_science_fair_model_comparison_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pywavefront

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.spatial import ConvexHull, distance

def calculate_volume(coordinates):
    # Calculate the volume of a convex hull formed by the coordinates
    # using the ConvexHull method from the scipy library
    hull = ConvexHull(coordinates)
    hull_volume = hull.volume

    return hull_volume

def compare_coordinates_metrics(coord_set1, coord_set2):
    volume1 = calculate_volume(coord_set1)
    volume2 = calculate_volume(coord_set2)
    # RMSE
    rmse = np.sqrt(mean_squared_error([volume1], [volume2]))
    # MAE
    mae = mean_absolute_error([volume1], [volume2])
    # VOI
    voi = 1 - (min(volume1, volume2) / max(volume1, volume2))
    # Jaccard Index
    intersection = np.intersect1d(volume1, volume2)
    union = np.union1d(volume1, volume2)
    jaccard_index = len(intersection) / len(union)
    # Hausdorff Distance
    hausdorff_distance = max(distance.directed_hausdorff(coord_set1, coord_set2)[0],
                             distance.directed_hausdorff(coord_set2, coord_set1)[0])
    print(f"Theoretical Volume 1: {volume1}")
    print(f"Theoretical Volume 2: {volume2}")
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Volume Overlap Index (VOI): {voi}")
    print(f"Jaccard Index: {jaccard_index}")
    print(f"Hausdorff Distance: {hausdorff_distance}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from scipy.spatial import ConvexHull, distance
from scipy.interpolate import griddata

def compare_coordinates_heat_volume(coord_set1, coord_set2, grid_resolution=100):
    volume1 = calculate_volume(coord_set1)
    volume2 = calculate_volume(coord_set2)
    x1, y1, z1 = np.array(coord_set1).T
    x2, y2, z2 = np.array(coord_set2).T
    x_min, x_max = min(np.min(x1), np.min(x2)), max(np.max(x1), np.max(x2))
    y_min, y_max = min(np.min(y1), np.min(y2)), max(np.max(y1), np.max(y2))
    z_min, z_max = min(np.min(z1), np.min(z2)), max(np.max(z1), np.max(z2))
    grid_x, grid_y, grid_z1 = np.meshgrid(
        np.linspace(x_min, x_max, grid_resolution),
        np.linspace(y_min, y_max, grid_resolution),
        np.linspace(z_min, z_max, grid_resolution),
        indexing='ij'
    )
    points1 = np.column_stack((x1, y1, z1))
    values1 = np.ones(len(coord_set1)) * volume1
    points2 = np.column_stack((x2, y2, z2))
    values2 = np.ones(len(coord_set2)) * volume2
    grid_volume1 = griddata(points1, values1, (grid_x, grid_y, grid_z1), method='linear')
    grid_volume2 = griddata(points2, values2, (grid_x, grid_y, grid_z1), method='linear')
    volume_difference = np.abs(grid_volume1 - grid_volume2)
    # Create a 3D heatmap
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(grid_x.flatten(), grid_y.flatten(), grid_z1.flatten(), c=volume_difference.flatten(), cmap='viridis', marker='o')
    # Customize the colorbar
    cbar = plt.colorbar(sc)
    cbar.set_label('Volume Difference', rotation=270, labelpad=15)
    ax.set_title('3D Heatmap - Volume Difference')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

In [ ]:
#load obj points
def load_obj(file_path):
    vertices = []

    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if parts:
                if parts[0] == 'v' and len(parts) == 4:
                    # Read vertices (v x y z)
                    x, y, z = map(float, parts[1:])
                    vertices.append((x, y, z))

    return vertices

In [ ]:
compare_coordinates_metrics(load_obj("/content/drive/MyDrive/Science Fair 2024 Data/patient 040179-12/040179-obj-2/040179_19011026_AXIALES_C+_msk/tinker_centered.obj"), list_of_coordinates2)

In [ ]:
compare_coordinates_heat_volume(load_obj("/content/drive/MyDrive/Science Fair 2024 Data/patient 040179-12/040179-obj-2/040179_19011026_AXIALES_C+_msk/tinker_centered.obj"), list_of_coordinates2)

In [ ]:
list_of_coordinates2 = []